In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1. Create dummy data
X = torch.tensor([[1.0, 2.0], [2.0, 1.0], [2.5, 2.5], [3.0, 1.0], [1.0, 3.0], [4.0, 5.0]], dtype=torch.float32)
y = torch.tensor([0, 0, 0, 1, 0, 1], dtype=torch.float32).unsqueeze(1)  # shape (N, 1)

# 2. Define the model
model = nn.Sequential(
    nn.Linear(2, 1),  # input: 2 features, output: 1
    nn.Sigmoid()
)

# 3. Define loss and optimizer
criterion = nn.BCELoss()  # Binary cross-entropy loss — good for binary classification.
optimizer = optim.SGD(model.parameters(), lr=0.1) # Stochastic Gradient Descent with learning rate 0.1.

# 4. Train the model
for epoch in range(100):
    y_pred = model(X) #Forward pass – get predictions.
    loss = criterion(y_pred, y) #Calculate loss between prediction and actual.

    optimizer.zero_grad() # Clear gradients from last step.
    loss.backward() # Backpropagate to compute gradients.
    optimizer.step() #Update model weights using gradients.

    if epoch % 10 == 0:
        print(f"Epoch {epoch} Loss: {loss.detach().item():.4f}") # Extract scalar loss (detached from computation graph) and save for plotting.

# 5. Test the model
with torch.no_grad():
    test = torch.tensor([[2.0, 2.0], [4.0, 4.0]], dtype=torch.float32)
    preds = model(test)
    print("Predictions:", preds)


Epoch 0 Loss: 0.7468
Epoch 10 Loss: 0.6904
Epoch 20 Loss: 0.6648
Epoch 30 Loss: 0.6411
Epoch 40 Loss: 0.6190
Epoch 50 Loss: 0.5984
Epoch 60 Loss: 0.5791
Epoch 70 Loss: 0.5611
Epoch 80 Loss: 0.5442
Epoch 90 Loss: 0.5283
Predictions: tensor([[0.4408],
        [0.5794]])


### Debugging
```
serWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /opt/pytorch/aten/src/ATen/native/Scalar.cpp:22.)
  print(f"Epoch {epoch} Loss: {loss.item():.4f}")
```
Explicitly detach the tensor before calling .item():


This happens when you call `.item()` directly on a tensor that tracks gradients (like `loss` in training). While harmless in logging, it's best to explicitly detach it.

---

### ✅ Fix

Replace this line:

```python
print(f"Epoch {epoch} Loss: {loss.item():.4f}")
````

with:
```
print(f"Epoch {epoch} Loss: {loss.detach().item():.4f}")
```

In [4]:
# Save model
torch.save(model.state_dict(), "/simple_model.pt")

In [5]:
# Load model for inference
model = nn.Sequential(
    nn.Linear(2, 1), # Takes 2 inputs (x, y) and outputs 1 value.
    nn.Sigmoid() # Squashes output to [0,1], representing probability of class 1.
)
model.load_state_dict(torch.load("simple_model.pt"))
model.eval()

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
  (1): Sigmoid()
)

In [6]:
# Predict new points
with torch.no_grad():
    test_points = torch.tensor([[2.0, 2.0], [4.0, 4.0]], dtype=torch.float32)
    preds = model(test_points)
    print("Predictions:", preds)
    print("Class labels:", (preds > 0.5).int())

Predictions: tensor([[0.4408],
        [0.5794]])
Class labels: tensor([[0],
        [1]], dtype=torch.int32)


In [ ]:
# plot it
